### Import packages and Gaia tables

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from astropy.table import Table, join

from astroquery.gaia import Gaia
#tables = Gaia.load_tables(only_names=True)
#for table in tables:
#    print(table.name)

### Load Gaia_source data

In [2]:
meta_source = Gaia.load_table('gaiadr3.gaia_source')
meta_source
print(meta_source)

meta_galaxies = Gaia.load_table('gaiadr3.galaxy_candidates')
print(meta_galaxies)

#meta_astrophys = Gaia.load.table('gaiadr3.astrophysical_parameters_supp')
#meta_astrophys
#print(meta_astrophys)

TAP Table name: gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Size (bytes): 3646930329600
Num. columns: 152
TAP Table name: gaiadr3.galaxy_candidates
Description: This table contains parameters derived from various modules dedicated to the classification and characterisation of sources considered as galaxy candidates. This table has been constructed with the intention to be complete rather than pure and, as such,

### List gaia source data

# Search Gaia DR3 host galaxy

In [3]:
ra = 272.464618
dec = -20.987513

In [4]:
id = "ZTF25acahpls"
ra = 272.464618
dec = -20.987513


In [5]:

query = f"""
SELECT
  DISTANCE(POINT('ICRS', {ra}, {dec}), POINT('ICRS', s.ra, s.dec)) AS ang_sep,
  s.source_id,
  s.ra, s.dec,
  s.ref_epoch,
  s.classprob_dsc_combmod_galaxy,
  gc.source_id AS galaxy_candidate_source_id
FROM gaiadr3.gaia_source AS s
LEFT JOIN gaiadr3.galaxy_candidates AS gc
  ON gc.source_id = s.source_id
WHERE 1 = CONTAINS(
  POINT('ICRS', s.ra, s.dec),
  CIRCLE('ICRS', {ra}, {dec}, 5./3600.)
)
ORDER BY ang_sep ASC
"""


In [6]:
job = Gaia.launch_job(query)
results = job.get_results()
results

ang_sep,source_id,ra,dec,ref_epoch,classprob_dsc_combmod_galaxy,galaxy_candidate_source_id
,,deg,deg,yr,,
float64,int64,float64,float64,float64,float32,int64
0.00021075697712312488,4094089236026720512,272.464788295336,-20.987374659063054,2016.0,5.0959643e-13,--
0.0009020367194338521,4094089240340869760,272.4638476815814,-20.98805742892604,2016.0,3.314698e-09,--
0.000915317700872032,4094089240323849216,272.46477900362373,-20.988415889697198,2016.0,--,--
0.0013254511998529825,4094089240323849472,272.46565301665777,-20.988420188051194,2016.0,3.224387e-08,--


In [7]:
# Convert Astropy Table to Pandas DataFrame
df = results.to_pandas()

# Convert ang_sep from degrees to arcseconds
df['ang_sep_arcsec'] = df['ang_sep'] * 3600.0

# Reorder and rename columns
df = df[['source_id', 'ang_sep_arcsec', 'ra', 'dec', 
         'classprob_dsc_combmod_galaxy', 'galaxy_candidate_source_id']]

# Display with limited precision for readability
pd.set_option('display.precision', 3)
display(id)
display(df)


'ZTF25acahpls'

,source_id,ang_sep_arcsec,ra,dec,classprob_dsc_combmod_galaxy,galaxy_candidate_source_id
0,4094089236026720512,0.759,272.465,-20.987,5.096e-13,<NA>
1,4094089240340869760,3.247,272.464,-20.988,3.315e-09,<NA>
2,4094089240323849216,3.295,272.465,-20.988,NaN,<NA>
3,4094089240323849472,4.772,272.466,-20.988,3.224e-08,<NA>
